In [1]:
import torch
from torch import nn

In [3]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

Using cpu


In [4]:
class FeedForward(nn.Module):
    def __init__(self):
        self.W1 = nn.Linear(512, 2048)
        self.W2 = nn.Linear(2048, 512)
        self.ffStack = nn.Sequential(
            self.W1(),
            nn.ReLU(),
            self.W2()
        )
    def forward(self, x):
        return self.ffStack(x)




